In [33]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key=os.getenv("GEMINI_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
generation_config = {
            "temperature": 0.2,
            "top_p": 0.9,
            "max_output_tokens": 4096,
            "response_mime_type": "application/json"
        }
llm=ChatGoogleGenerativeAI(model="gemini-pro",generation_config=generation_config)
embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [35]:
from langchain_community.document_loaders import WebBaseLoader

load=WebBaseLoader("https://www.falkordb.com/blog/advanced-rag/")
data=load.load()

In [36]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
docs=text.split_documents(data)

In [37]:
from langchain_community.vectorstores import FAISS
vector_stores=FAISS.from_documents(documents=docs,embedding=embedding)
retriever=vector_stores.as_retriever(search_type="similarity",search_kwargs={"k":3})
vector_stores.save_local("smote_index")


In [ ]:
from langchain.prompts import ChatPromptTemplate #HYDE LOGİC 
template=""" 
Given the question: {question}, write a detailed and informative passage that provides an answer, explanation, or context about the topic. Be specific and concise, 
focusing on relevant facts, examples, and insights. The response should resemble an excerpt from an article, book, or scholarly discussion.
"""
prompt_hyde=ChatPromptTemplate.from_template(template)
user_question="What is Self-Query Retrieval in rag ?"
query=prompt_hyde.format(question=user_question)  #format provide replacing your variable into the variable in template. 'question' is variable in template 'user question' is your variable out of template . it has been replaced
query

'Human:  \nGiven the question: What is Self-Query Retrieval in rag ?, write a detailed and informative passage that provides an answer, explanation, or context about the topic. Be specific and concise, \nfocusing on relevant facts, examples, and insights. The response should resemble an excerpt from an article, book, or scholarly discussion.\n'

In [39]:
hypothetical_answer=llm.invoke(query).content
hypothetical_answer

"Self-Query Retrieval in RAG (Self-Query Retrieval with Retrieval-Augmented Generation) is a novel retrieval-based approach to question answering introduced by Lewis et al. (2020). Unlike traditional question answering systems that rely solely on document retrieval or generative language models, Self-Query Retrieval combines both approaches to achieve superior performance.\n\nThe key idea behind Self-Query Retrieval is to use a pretrained retrieval model to identify relevant documents for a given question. These documents are then used to generate a self-query, which is a reformulation of the original question that is more specific and informative. The self-query is then used to retrieve a new set of documents, which are used to generate the final answer.\n\nSelf-Query Retrieval has several advantages over traditional question answering systems. First, it allows the system to leverage the strengths of both retrieval and generative models. Second, it enables the system to handle complex

In [40]:
#get relevant documents from document based on output from llm 
relevant_docs=retriever.invoke(hypothetical_answer)
relevant_docs


[Document(metadata={'source': 'https://www.falkordb.com/blog/advanced-rag/', 'title': 'Advanced RAG Techniques: What They Are & How to Use Them', 'description': 'Master advanced RAG techniques to enhance AI performance, accuracy, and efficiency. Learn methods for optimizing retrieval and generation in complex queries.', 'language': 'en-US'}, page_content='Self-RAG is an advanced technique that empowers your system to refine its own retrieval and generation process by iterating on its outputs. In Self-RAG, the model doesn’t just rely on the initial retrieval but actively re-evaluates and adjusts its approach by generating follow-up queries and responses. This iterative process allows the model to correct its own mistakes, fill in gaps, and enhance the quality of the final output.You can think of Self-RAG as your model’s ability to self-correct'),
 Document(metadata={'source': 'https://www.falkordb.com/blog/advanced-rag/', 'title': 'Advanced RAG Techniques: What They Are & How to Use The

In [50]:
relevant_texts=["/n".join(doc.page_content for doc in relevant_docs)]
relevant_texts

['Self-RAG is an advanced technique that empowers your system to refine its own retrieval and generation process by iterating on its outputs. In Self-RAG, the model doesn’t just rely on the initial retrieval but actively re-evaluates and adjusts its approach by generating follow-up queries and responses. This iterative process allows the model to correct its own mistakes, fill in gaps, and enhance the quality of the final output.You can think of Self-RAG as your model’s ability to self-correct/nSelf-RAG as your model’s ability to self-correct and improve its answers. By generating an initial response, evaluating its accuracy and relevance, and then adjusting the retrieval process accordingly, the model can produce more nuanced and accurate answers. This approach is particularly useful in scenarios where a single round of retrieval might not be sufficient to provide the best possible answer. With Self-RAG, you enable your system to be more adaptive and precise./nRetrieval-Augmented Gene

In [ ]:


template="""  
answer the following question in detailed based on context:
context:{context}
question:{question}
"""
prompt=ChatPromptTemplate.from_template(template)


In [47]:
query=prompt.format(context=relevant_texts,question=user_question) ##this response is taken by hyde technique
response=llm.invoke(query)
answer=response.content


In [52]:
print(answer)

Self-Query Retrieval (Self-RAG) is an advanced technique in Retrieval-Augmented Generation (RAG) that enables a system to refine its own retrieval and generation process iteratively. Unlike basic RAG, where the model relies solely on the initial retrieval, Self-RAG involves the model actively re-evaluating and adjusting its approach by generating follow-up queries and responses. This iterative process allows the model to correct its own mistakes, fill in gaps, and enhance the quality of the final output.

In Self-RAG, the model generates an initial response, evaluates its accuracy and relevance, and then adjusts the retrieval process accordingly. This approach is particularly useful in scenarios where a single round of retrieval might not be sufficient to provide the best possible answer. By enabling the model to self-correct and improve its answers, Self-RAG contributes to more nuanced and accurate responses.


In [53]:
# this response is taken  by basic rag implementation
relevant_docs2=retriever.invoke(user_question)
relevant_texts2=["/n".join(doc.page_content for doc in relevant_docs2)]
query2=template.format(context=relevant_texts2,question=user_question)

In [54]:
response=llm.invoke(query2)
answer=response.content


In [55]:
print(answer)

Self-Query Retrieval (Self-RAG) is an advanced technique used in Retrieval-Augmented Generation (RAG) systems. It allows the model to iteratively refine its retrieval and generation process by generating follow-up queries and responses.

In Self-RAG, the model doesn't rely solely on the initial retrieval but actively re-evaluates and adjusts its approach. This iterative process enables the model to:

1. Correct its own mistakes by identifying and addressing inconsistencies or errors in the initial retrieval.

2. Fill in gaps by retrieving additional relevant documents or knowledge snippets that were not captured in the initial retrieval.

3. Enhance the quality of the final output by synthesizing the retrieved information more effectively and generating more nuanced and accurate responses.

Self-RAG empowers the system to be more adaptive and precise by enabling it to self-correct and improve its answers. This is particularly useful in scenarios where a single round of retrieval might 

In [42]:
"""from dotenv import load_dotenv
import os
load_dotenv()
api_key=os.getenv("GEMINI_API_KEY")
from langchain_google_genai import GoogleGenerativeAI,ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
llm=ChatGoogleGenerativeAI(model="gemini-pro",convert_system_message_to_human=True,api_key=api_key)
embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001",)
from langchain_community.document_loaders import PyPDFLoader

load=PyPDFLoader("2201.08528v3.pdf")
data=load.load()
from langchain_text_splitters import RecursiveCharacterTextSplitter
text=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
docs=text.split_documents(data)
from langchain_community.vectorstores import FAISS
vector_stores=FAISS.from_documents(documents=docs,embedding=embedding)
retriever=vector_stores.as_retriever(search_type="similarity",search_kwargs={"k":3})"""

'from dotenv import load_dotenv\nimport os\nload_dotenv()\napi_key=os.getenv("GEMINI_API_KEY")\nfrom langchain_google_genai import GoogleGenerativeAI,ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings\nllm=ChatGoogleGenerativeAI(model="gemini-pro",convert_system_message_to_human=True,api_key=api_key)\nembedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001",)\nfrom langchain_community.document_loaders import PyPDFLoader\n\nload=PyPDFLoader("2201.08528v3.pdf")\ndata=load.load()\nfrom langchain_text_splitters import RecursiveCharacterTextSplitter\ntext=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)\ndocs=text.split_documents(data)\nfrom langchain_community.vectorstores import FAISS\nvector_stores=FAISS.from_documents(documents=docs,embedding=embedding)\nretriever=vector_stores.as_retriever(search_type="similarity",search_kwargs={"k":3})'